In [ ]:
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix, quaternion_yaw
import numpy as np
import pandas as pandas
from tqdm import tqdm, tqdm_notebook
import os
import sys





In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!python3.8 --version

In [ ]:
ARTIFACTS_FOLDER = './lyft_custom_artifacts'
LYFT_DATASET_ROOT = '/kaggle/input/3d-object-detection-for-autonomous-vehicles'
level5data = LyftDataset(data_path=DATA_PATH, json_path= os.path.join(DATA_PATH, 'train_data'))
os.makedirs(ARTIFACTS_FOLDER, exist_ok = True)
classes = ['car', 'motorcycle', 'bus', 'bicycle', 'truck', 'pedestrian', 'other_vehicle', 'animal', 'emergency_vehicle']

In [ ]:
level5data.sample

In [ ]:
TARGET_CLASSES = ['truck']
filtered_sample_token = []
for sample in tqdm_notebook(level5data.sample):
    sample_lidar_token = sample['data']['LIDAR_TOP']
    boxes = level5data.get_boxes(sample_lidar_token)
    for box in boxes:
        if box.name in TARGET_CLASSES:
            filtered_sample_token.append(sample['token'])
            break

In [ ]:
train_samples = filtered_sample_token[:round(len(filtered_sample_token)*0.8)]
valid_samples = filtered_sample_token[round(len(filtered_sample_token)*0.8):]
len(valid_samples)

## train/test data preprocess

In [ ]:
len(train_samples)

In [ ]:
len(valid_samples)

In [ ]:
level5data.get('sample', train_samples[0])

In [ ]:
train_data_folder = os.path.join(ARTIFACTS_FOLDER, "samples_train")
valid_data_folder = os.path.join(ARTIFACTS_FOLDER, "samples_valid")
for samples, folder in [(train_samples, train_data_folder), (valid_samples, valid_data_folder)]:
    os.makedir(folder, exist_ok = True)
    for sample_token in tqdm_notebook(samples):
        sample = level5data.get('sample', sample_token)
        

In [ ]:
sys.path.append('/kaggle/code/lyft-tho')
from second.data.dataset import Dataset, register_dataset
LYFT_DATASET_ROOT = '/kaggle/input/3d-object-detection-for-autonomous-vehicles'

# @register_dataset
class CustomLyftDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path=LYFT_DATASET_ROOT, info_path=None,
                 class_names=None, prep_func=None,
                 num_point_features=None):

        data_dir = root_path
        json_dir = os.path.join(root_path, 'train_data')
        self.class_names = class_names
        self.lyft = LyftDataset(data_path=data_dir, json_path=json_dir)
        self._prep_func = prep_func

        self.filtered_sample_tokens = []
        for sample in self.lyft.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.lyft.get_boxes(sample_lidar_token)
            for box in boxes:
                if box.name in self.class_names:
                    self.filtered_sample_tokens.append(sample_token)
                    break

        self.split = np.arange(len(self.filtered_sample_tokens))

    def __len__(self):
        return self.split.shape[0]

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        try:
            example = self._prep_func(input_dict=input_dict)
            return example
        except:
            return input_dict

    def get_sensor_data(self, query):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None,
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query]
            }
        }
        points = self.getPoints(query)
        boxes_dict = self.getBoxes(query)

        res['lidar']['points'] = points

        gt_boxes = []
        gt_names = []

        for box in boxes_dict:
            xyz = box.center
            wlh = box.wlh
            theta = quaternion_yaw(box.orientation)
            gt_boxes.append([xyz[0], xyz[1], xyz[2], wlh[0], wlh[1], wlh[2], -theta - np.pi / 2])
            gt_names.append(box.name)

        res['lidar']['annotation'] = {
            'boxes': gt_boxes,
            'names': gt_names,
        }
        return res

        ###

    def getPoints(self, index):
        sample = self.lyft.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.lyft.get('sample_data', sample_lidar_token)
        ego_pose = self.lyft.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.lyft.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.lyft, sample, 'LIDAR_TOP',
                                                                           'LIDAR_TOP', num_sweeps=10)
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"Failed to load Lidar Pointcloud for {sample}:{e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5

        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]

        return points_cat

    def getBoxes(self, index):

        sample = self.lyft.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.lyft.get('sample_data', sample_lidar_token)
        ego_pose = self.lyft.get('ego_pose', lidar_data['ego_pose_token'])

        boxes_dict = self.lyft.get_boxes(sample_lidar_token)

        keep_box_idx = []
        for i, box in enumerate(boxes_dict):
            if box.name in self.class_names:
                keep_box_idx.append(i)

        boxes_dict = [box for i, box in enumerate(boxes_dict) if i in keep_box_idx]

        return boxes_dict

    def move_boxes_to_car_space(self,boxes, ego_pose):
        """
        Move boxes from world space to car space.
        Note: mutates input boxes.
        """
        translation = -np.array(ego_pose['translation'])
        rotation = Quaternion(ego_pose['rotation']).inverse

        for box in boxes:
            # Bring box to car space
            box.translate(translation)
            box.rotate(rotation)
        
    

In [ ]:
a = CustomLyftDataset(class_names = 'truck')

In [ ]:
a[5]

In [ ]:
level5data.render_sample_data(
    level5data.get('sample',a[5]['metadata']['token'])['data']['LIDAR_TOP'])

In [ ]:
# from lyft_dataset_sdk.lyftdataset import LyftDataset
# from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
import sys
sys.path.append('/kaggle/code/lyft-tho')

from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix, quaternion_yaw
from second.data.dataset import Dataset, register_dataset
from nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, Box, Quaternion
from nuscenes.utils.geometry_utils import *
import numpy as np
import pandas as pandas
from tqdm import tqdm, tqdm_notebook
import os


LYFT_DATASET_ROOT = '/kaggle/input/3d-object-detection-for-autonomous-vehicles'



In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
import sys
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pandas
sys.path.append('/kaggle/code/lyft-tho')
import fire
import pickle
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix, quaternion_yaw
from second.data.dataset import Dataset, register_dataset,get_dataset_class
from nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, Box, Quaternion
from nuscenes.utils.geometry_utils import *

VERSION = 'trainval'
NUSC_DATASET_ROOT = f'/media/starlet/LdTho/data/sets/nuscenes/v1.0-{VERSION}'
OBJECT_CLASSES = ['animal',
                  'human.pedestrian.adult',
                  'human.pedestrian.child',
                  'human.pedestrian.construction_worker',
                  'human.pedestrian.personal_mobility',
                  'human.pedestrian.police_officer',
                  'human.pedestrian.stroller',
                  'human.pedestrian.wheelchair',
                  'movable_object.barrier',
                  'movable_object.debris',
                  'movable_object.pushable_pullable',
                  'movable_object.trafficcone',
                  'static_object.bicycle_rack',
                  'vehicle.bicycle',
                  'vehicle.bus.bendy',
                  'vehicle.bus.rigid',
                  'vehicle.car',
                  'vehicle.construction',
                  'vehicle.emergency.ambulance',
                  'vehicle.emergency.police',
                  'vehicle.motorcycle',
                  'vehicle.trailer',
                  'vehicle.truck']

# @register_dataset
class CustomNuscDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path=NUSC_DATASET_ROOT, info_path=None,
                 class_names=["movable_object.trafficcone"], prep_func=None,
                 num_point_features=None):

        data_dir = root_path
        json_dir = os.path.join(root_path, f'v1.0-{VERSION}')
        print(json_dir)
        self.class_names = class_names
        self.nusc = NuScenes(dataroot=data_dir,version=f'v1.0-{VERSION}')
        self._prep_func = prep_func
        self.box_classes = set()
        self.filtered_sample_tokens = []
        for sample in self.nusc.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.nusc.get_boxes(sample_lidar_token)
            for box in boxes:
                self.box_classes.add(box.name)
                if box.name in self.class_names:
                    self.filtered_sample_tokens.append(sample_token)
                    break

        self.split = np.arange(len(self.filtered_sample_tokens))

    def __len__(self):
        return self.split.shape[0]

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        try:
            example = self._prep_func(input_dict=input_dict)
            return example
        except:
            return input_dict
    def __name__(self):
        return "CustomNuscDataset"
    def get_sensor_data(self, query):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None,
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query]
            }
        }
        points = self.getPoints(query)
        boxes_dict = self.getBoxes(query)

        res['lidar']['points'] = points

        gt_boxes = []
        gt_names = []

        for box in boxes_dict:
            xyz = box.center
            wlh = box.wlh
            theta = quaternion_yaw(box.orientation)
            gt_boxes.append([xyz[0], xyz[1], xyz[2], wlh[0], wlh[1], wlh[2], -theta - np.pi / 2])
            gt_names.append(box.name)

        res['lidar']['annotations'] = {
            'boxes': np.asarray(gt_boxes,dtype=np.float32),
            'names': gt_names,
        }
        return res

        ###

    def getPoints(self, index):
        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.nusc, sample, 'LIDAR_TOP',
                                                                           'LIDAR_TOP')
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"Failed to load Lidar Pointcloud for {sample}:{e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5

        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]

        return points_cat

    def getBoxes(self, index):

        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])

        boxes_dict = self.nusc.get_boxes(sample_lidar_token)

        keep_box_idx = []
        for i, box in enumerate(boxes_dict):
            if box.name in self.class_names:
                keep_box_idx.append(i)

        boxes_dict = [box for i, box in enumerate(boxes_dict) if i in keep_box_idx]

        return boxes_dict

    def move_boxes_to_car_space(self,boxes, ego_pose):
        """
        Move boxes from world space to car space.
        Note: mutates input boxes.
        """
        translation = -np.array(ego_pose['translation'])
        rotation = Quaternion(ego_pose['rotation']).inverse

        for box in boxes:
            # Bring box to car space
            box.translate(translation)
            box.rotate(rotation)

In [ ]:
train_data = CustomNuscDataset()

In [ ]:
train_data[3]

In [ ]:
with open('/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval/kitti_dbinfos_train.pkl', 'rb') as infile:
    train = pickle.load(infile)
train.keys()

In [ ]:
train['movable_object.trafficcone'][0]

In [ ]:
[i for i in train['movable_object.trafficcone'] if i['path'] == 'gt_database/0_movable_object.trafficcone_0.bin']

In [ ]:
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pandas
import sys
sys.path.append('/kaggle/code/ConeDetectionPointpillars')
import fire
import pickle
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix, quaternion_yaw
from second.data.dataset import Dataset, register_dataset, get_dataset_class
from nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, Box, Quaternion
from nuscenes.utils.geometry_utils import *
from second.core import box_np_ops
VERSION = 'trainval'
TRAINVAL_SPLIT_PERCENTAGE = 0.999 if VERSION == 'trainval' else 0.8
NameMapping = {
    'movable_object.barrier': 'barrier',
    'vehicle.bicycle': 'bicycle',
    'vehicle.bus.bendy': 'bus',
    'vehicle.bus.rigid': 'bus',
    'vehicle.car': 'car',
    'vehicle.construction': 'construction_vehicle',
    'vehicle.motorcycle': 'motorcycle',
    'human.pedestrian.adult': 'pedestrian',
    'human.pedestrian.child': 'pedestrian',
    'human.pedestrian.construction_worker': 'pedestrian',
    'human.pedestrian.police_officer': 'pedestrian',
    'movable_object.trafficcone': 'traffic_cone',
    'vehicle.trailer': 'trailer',
    'vehicle.truck': 'truck',
    'movable_object.pushable_pullable': 'DontCare',
    'movable_object.debris': 'DontCare'
}
DefaultAttribute = {
    "car": "vehicle.parked",
    "pedestrian": "pedestrian.moving",
    "trailer": "vehicle.parked",
    "truck": "vehicle.parked",
    "bus": "vehicle.parked",
    "motorcycle": "cycle.without_rider",
    "construction_vehicle": "vehicle.parked",
    "bicycle": "cycle.without_rider",
    "barrier": "",
    "traffic_cone": "",
}


# @register_dataset
class CustomNuscDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path=f'/media/starlet/LdTho/data/sets/nuscenes/v1.0-{VERSION}', info_path=None,
                 class_names=["traffic_cone"], prep_func=None,
                 num_point_features=None):
        self.NumPointFeatures = 5
        self.class_names = class_names
        self.nusc = NuScenes(dataroot=root_path, version=f'v1.0-{VERSION}')
        self._prep_func = prep_func
        self.filtered_sample_tokens = []
        for sample in self.nusc.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.nusc.get_boxes(sample_lidar_token)
            box_names = [NameMapping[b.name] for b in boxes if b.name in NameMapping.keys()]
            for box in boxes:
                if box.name not in NameMapping.keys():
                    continue
                # if NameMapping[box.name] in self.class_names:
                if (NameMapping[box.name] in ["traffic_cone"]) & (box_names.count('traffic_cone') > 8):
                    self.filtered_sample_tokens.append(sample_token)
                    break
        self.filtered_sample_tokens = self.filtered_sample_tokens[
                                      :round(len(self.filtered_sample_tokens) * TRAINVAL_SPLIT_PERCENTAGE)]

        self.split = np.arange(len(self.filtered_sample_tokens))

    def __len__(self):
        return self.split.shape[0]

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        example = self._prep_func(input_dict=input_dict)
        example["metadata"] = input_dict["metadata"]
        if "anchors_mask" in example:
            example["anchors_mask"] = example["anchors_mask"].astype(np.uint8)
        return example

    def get_sensor_data(self, query, token = None):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None,
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query]
            }
        }
        if token:
            query = self.filtered_sample_tokens.index(token)
        points = self.getPoints(query)
        boxes_dict = self.getBoxes(query)

        res['lidar']['points'] = points

        gt_boxes = []
        gt_names = []

        for box in boxes_dict:
            xyz = box.center
            wlh = box.wlh
            theta = quaternion_yaw(box.orientation)
            gt_boxes.append([xyz[0], xyz[1], xyz[2], wlh[0], wlh[1], wlh[2], -theta - np.pi / 2])
            gt_names.append(box.name)
        gt_boxes = np.concatenate(gt_boxes).reshape(-1, 7)
        gt_names = np.array(gt_names)
        res['lidar']['annotations'] = {
            'boxes': gt_boxes,
            'names': gt_names,
        }
        return res

        ###

    def getPoints(self, index):
        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.nusc, sample, 'LIDAR_TOP',
                                                                           'LIDAR_TOP')
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"Failed to load Lidar Pointcloud for {sample}:{e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5

        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]

        return points_cat

    def getBoxes(self, index):

        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])

        boxes_dict = self.nusc.get_boxes(sample_lidar_token)

        keep_box_idx = []
        for i, box in enumerate(boxes_dict):
            if box.name not in NameMapping.keys():
                continue
            if NameMapping[box.name] in self.class_names:
                box.name = NameMapping[box.name]
                keep_box_idx.append(i)

        boxes_dict = [box for i, box in enumerate(boxes_dict) if i in keep_box_idx]
        self.move_boxes_to_car_space(boxes_dict, ego_pose)
        # print(boxes_dict)
        return boxes_dict

    def move_boxes_to_car_space(self, boxes, ego_pose):
        """
        Move boxes from world space to car space.
        Note: mutates input boxes.
        """
        translation = -np.array(ego_pose['translation'])
        rotation = Quaternion(ego_pose['rotation']).inverse

        for box in boxes:
            # Bring box to car space
            box.translate(translation)
            box.rotate(rotation)


# @register_dataset
class CustomNuscTestDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path=f'/media/starlet/LdTho/data/sets/nuscenes/v1.0-{VERSION}',
                 info_path=None,
                 class_names=['traffic_cone'],
                 prep_func=None,
                 num_point_features=None,
                 multi_test=False):
        print(root_path)
        self.nusc = NuScenes(dataroot=root_path, version=f'v1.0-{VERSION}')
        self.class_names = class_names
        self._prep_func = prep_func
        self.filtered_sample_tokens = []
        self.multi_test = multi_test
        for sample in self.nusc.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.nusc.get_boxes(sample_lidar_token)
            for box in boxes:
                # self.box_classes.add(box.name
                if box.name not in NameMapping.keys():
                    continue
                if NameMapping[box.name] in self.class_names:
                    self.filtered_sample_tokens.append(sample_token)
                    break
        self.filtered_sample_tokens = self.filtered_sample_tokens[
                                      round(len(self.filtered_sample_tokens) * TRAINVAL_SPLIT_PERCENTAGE):]
        self.split = np.arange(len(self.filtered_sample_tokens))
        self.num_samples = len(self.filtered_sample_tokens)
        self.rot = 0.0
        self.scale = 1.0

    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        example = self._prep_func(input_dict=input_dict)
        example["metadata"] = input_dict["metadata"]
        if "anchors_mask" in example:
            example["anchors_mask"] = example["anchors_mask"].astype(np.uint8)
        return example

    def get_sensor_data(self, query):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query],
            }
        }
        points = self.getPoints(query)
        res['lidar']['points'] = points
        return res

    def getPoints(self, query):
        sample = self.nusc.get('sample', self.filtered_sample_tokens[query])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.nusc, sample, 'LIDAR_TOP', 'LIDAR_TOP')
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"failed to load pointcloud for {sample}: {e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5
        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]
        return points_cat

    def evaluation(self, detections, output_dir):
        res_custom_nusc = self.evaluation_custom_nusc(detections, output_dir)
        res = {
            "results":{
                "nusc": res_custom_nusc["results"]["nusc"],
            },
            "details":{
                "eval.nusc": res_custom_nusc["detail"]["nusc"]
            }
        }
        return res
    def evaluation_custom_nusc(self, detections, output_dir):
        pass


# should shorten by inherit from parent class CustomNuscDataset
# @register_dataset
class CustomNuscEvalDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path=f'/media/starlet/LdTho/data/sets/nuscenes/v1.0-mini', info_path=None,
                 class_names=["traffic_cone"], prep_func=None,
                 num_point_features=None):
        self.NumPointFeatures = 5
        self.class_names = class_names
        self.nusc = NuScenes(dataroot=root_path, version=f'v1.0-mini')
        self._prep_func = prep_func
        self.filtered_sample_tokens = []
        for sample in self.nusc.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.nusc.get_boxes(sample_lidar_token)
            box_names = [b.name for b in boxes if b.name in NameMapping.keys()]
            for box in boxes:
                # self.box_classes.add(box.name
                if box.name not in NameMapping.keys():
                    continue
                # if NameMapping[box.name] in self.class_names:
                if (NameMapping[box.name] in ["traffic_cone"]) & (box_names.count('traffic_cone') > 7):
                    print(box_names)
                    self.filtered_sample_tokens.append(sample_token)
                    break
        self.filtered_sample_tokens = self.filtered_sample_tokens[
                                      :round(len(self.filtered_sample_tokens) * TRAINVAL_SPLIT_PERCENTAGE)]

        self.split = np.arange(len(self.filtered_sample_tokens))

    def __len__(self):
        return self.split.shape[0]

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        example = self._prep_func(input_dict=input_dict)
        example["metadata"] = input_dict["metadata"]
        if "anchors_mask" in example:
            example["anchors_mask"] = example["anchors_mask"].astype(np.uint8)
        return example

    def get_sensor_data(self, query, token = None):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None,
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query]
            }
        }
        if token:
            query = self.filtered_sample_tokens.index(token)
        points = self.getPoints(query)
        boxes_dict = self.getBoxes(query)

        res['lidar']['points'] = points

        gt_boxes = []
        gt_names = []

        for box in boxes_dict:
            xyz = box.center
            wlh = box.wlh
            theta = quaternion_yaw(box.orientation)
            gt_boxes.append([xyz[0], xyz[1], xyz[2], wlh[0], wlh[1], wlh[2], -theta - np.pi / 2])
            gt_names.append(box.name)
        gt_boxes = np.concatenate(gt_boxes).reshape(-1, 7)
        gt_names = np.array(gt_names)
        res['lidar']['annotations'] = {
            'boxes': gt_boxes,
            'names': gt_names,
        }
        return res

        ###

    def getPoints(self, index):
        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.nusc, sample, 'LIDAR_TOP',
                                                                           'LIDAR_TOP')
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"Failed to load Lidar Pointcloud for {sample}:{e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5

        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]

        return points_cat

    def getBoxes(self, index):

        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])

        boxes_dict = self.nusc.get_boxes(sample_lidar_token)

        keep_box_idx = []
        for i, box in enumerate(boxes_dict):
            if box.name not in NameMapping.keys():
                continue
            if NameMapping[box.name] in self.class_names:
                box.name = NameMapping[box.name]
                keep_box_idx.append(i)

        boxes_dict = [box for i, box in enumerate(boxes_dict) if i in keep_box_idx]
        self.move_boxes_to_car_space(boxes_dict, ego_pose)
        # print(boxes_dict)
        return boxes_dict

    def move_boxes_to_car_space(self, boxes, ego_pose):
        """
        Move boxes from world space to car space.
        Note: mutates input boxes.
        """
        translation = -np.array(ego_pose['translation'])
        rotation = Quaternion(ego_pose['rotation']).inverse

        for box in boxes:
            # Bring box to car space
            box.translate(translation)
            box.rotate(rotation)

# if __name__ == '__main__':
#     fire.Fire()
    # train_data = CustomNuscDataset()
    # test_data = CustomNuscTestDataset(root_path='/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval',
    #                                   )
#     print(train_data[1])


In [ ]:
test_dataset = CustomNuscDataset(root_path = '/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval')

In [ ]:
len(test_dataset)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
w, l, h = 0,0,0

for i in tqdm_notebook(range(len(test_dataset))):
    for box in test_dataset[i]['lidar']['annotations']['boxes']:
        w = max(w, box[3])
        l = max(l, box[4])
        h = max(h, box[5])
print(w,l,h)

In [ ]:
w, l, h = [],[],[]
x, y, z = [],[],[]
for i in tqdm_notebook(range(len(test_dataset))):
    for box in test_dataset[i]['lidar']['annotations']['boxes']:
        x.append(box[0])
        y.append(box[1])
        z.append(box[3])       
        w.append(box[3])
        l.append(box[4])
        h.append(box[5])
# print(w,l,h)

In [ ]:
test_dataset[i]['lidar']['annotations']

In [ ]:
plt.hist(x)

In [ ]:

plt.hist(y)

In [ ]:
plt.hist(z)

In [ ]:
plt.hist(w)

In [ ]:
plt.hist(l)

In [ ]:
plt.hist(h)